In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
from models import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from fastai.conv_learner import *
from fastai.dataset import *

In [5]:
import numpy as np

In [ ]:
!ln -s ~/KITTI KITTI

In [ ]:
ls

In [ ]:
ls KITTI/2011_10_03/2011_10_03_drive_0034_sync/image_03/data/0000003783.png

In [6]:
class UnsupFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        super().__init__(fnames, transform, path)
    def get_y(self, i): return None
    def get_c(self): return 0

In [7]:
train, val= pd.read_csv('./KITTI/training.csv'), pd.read_csv('./KITTI/validation.csv')

In [8]:
trn0, trn1, trn2 = train.t0, train.t1,train.t2

In [9]:
val0, val1, val2 = val.t0, val.t1, val.t2

val_idxs = get_cv_idxs(len(fnames), val_pct=min(0.01/keep_pct, 0.1))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), np.array(fnames))
len(val_x),len(trn_x)

In [10]:
aug_tfms = [
    #RandomDihedral(tfm_y=TfmType.NO),
    RandomLighting(b=0.05, c=0.05)]

In [11]:
arch = f
scale,bs = 1, 4
sz = 256 #np.array([345, 1242]) 
sz_y = sz // scale 

# scale,bs = 4,32

PATH = 'Fastai_TRN'

In [12]:
def get_MD(trn, val):
    tfms = tfms_from_model(
        arch,
        sz,
        crop_type=CropType.NO,
        tfm_y=TfmType.NO,
        aug_tfms=aug_tfms,
        sz_y=sz_y)
    datasets = ImageData.get_ds(
        UnsupFilesDataset,
        (trn, None),
        (val, None),
        tfms, path='KITTI/')
    md = ImageData(
        PATH,
        datasets,
        bs, num_workers=16,
        classes=None)
    
    return md

In [13]:
MD1 = get_MD(trn0, val0)
MD2 = get_MD(trn1, val1)
MD3 = get_MD(trn2, val2)

In [14]:
class Loss(nn.Module):
    def __init__(self, scale=0.01, Tscale=0.01):
        super().__init__()
        self.appr = TriAppearanceLoss()
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1, d2, d3, poses_x2, x1, x2, x3):
        return self.appr(d1, d3, poses_x2, x1, x2, x3)+scale * self.smooth(d1, d2, d3)
        

In [41]:
m = TriDepth(get_base(), 1).cuda()

In [48]:
l = Loss().cuda() 

In [43]:
opt = optim.Adam(m.parameters())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
imgbatch = next(iter(MD1.trn_dl))

img = MD1.trn_ds.denorm(imgbatch)[0]
plt.imshow(img); img.shape

In [ ]:
DL1, DL2, DL3 = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl)
imgs1, imgs2, imgs3= V([next(DL1), next(DL2),next(DL3)])

In [34]:
oo= torch.zeros([1,1]).type(torch.FloatTensor).cuda()
Eeye=torch.eye(3).unsqueeze(0).type(torch.FloatTensor).cuda()
def factorize(vecs, dim):
    mags = vecs.norm(p=2, dim=dim, keepdim=True)
    return vecs/mags, mags

def rot_vec2mat(rot_vecs):
    batch, _ = rot_vecs.size()
    directs, angles = factorize(rot_vecs, 1)

    K0 = directs[:,:1]
    K1 = directs[:,1:2]
    K2 = directs[:,2:]

    o = oo.repeat(batch, 1)
    eye = Eeye.repeat(batch, 1, 1)

    # print(K0.type, K2.type, K1.type, o.type, eye.type)

    K = torch.cat(seq=(o, -K2, K1, K2, o, K0, -K1, K0, o), dim=1).view(-1, 3, 3)
    
    #pdb.set_trace()
    angles = angles.unsqueeze(-1)
    res1 = K * angles.sin()
    res2 = torch.bmm(K,K) * (1-angles.cos())
    print(res1.size(), res2.size(), eye.size())
    return eye + res1 + res2

In [49]:
l.appr.offset.o.type()

'torch.cuda.FloatTensor'

In [52]:
xy = torch.meshgrid([torch.arange(10), torch.arange(20)])

AttributeError: module 'torch' has no attribute 'meshgrid'

In [51]:
offset1 = l.appr.offset.forward( poses_x2[:,0], d1 )

NameError: name 'xy' is not defined

In [18]:
DL1, DL2, DL3 = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl)

In [ ]:
x = torch.randn(2, 3)
torch.cat((x,x,x),1)

In [44]:
opt.zero_grad()
imgs1, imgs2, imgs3= V([next(DL1), next(DL2),next(DL3)])

d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)
loss = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3)
loss.backward()
opt.step()

TypeError: cat received an invalid combination of arguments - got (dim=int, seq=tuple, ), but expected one of:
 * (sequence[torch.FloatTensor] seq)
 * (sequence[torch.FloatTensor] seq, int dim)
      didn't match because some of the arguments have invalid types: ([31;1mseq=tuple[0m, [32;1mdim=int[0m, )


In [ ]:
def fit(model, metric, opt, MD1, MD2, MD3):
    DL1, DL2, DL3 = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl)
    losses = []
    for i in tqdm(range(len(MD1.trn_ds))):
        opt.zero_grad()
        imgs1, imgs2, imgs3= V([next(DL1), next(DL2),next(DL3)])
        
        d1, d2, d3, poses_x2 = model(imgs1, imgs2, imgs3)
        loss = metric(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3)
        loss.backward()
        opt.step()
        losses.append(loss)
    return torch.cat(loesses).mean()

In [ ]:
a=torch.randn(1,2,6); a

In [ ]:
b=a[:,0]

In [ ]:
b[:,:3]

In [ ]:
fit(m, l, opt, MD1, MD2, MD3)